# Exemple wikipedia
Chope plusieurs articles de manière async

In [ ]:
import asyncio, aiohttp

async def fetch(session, url):
	async with session.get(url) as response:
		return await response.text()

async def fetch_all(urls):
	async with aiohttp.ClientSession() as session:
		texts = await asyncio.gather(*[
			fetch(session, url)
			for url in urls
		])
		#print(texts)
		print("fetched successfully !")
		return texts

urls = [f'https://en.wikipedia.org/wiki/{year}' for year in range(1990, 2020)]

async def main():
	await asyncio.gather(fetch_all(urls))

await(main())

# Emulation de la req du navigateur
On veut émuler la req du navigateur pour avoir directement le .json du(des) contenu(s) qui nous intéresse(nt)
Il faut définir les headers et paramètres pour ensuite faire une req.  
On peut changer un paramètre à chaque requête pour avoir les données suivantes  
Problème : le site nous rate-limit  
Solution : Changer les headers à chaque req, intervalles de temps aléatoires entre req,  
Set un cookie, rotating proxies...

In [ ]:
import asyncio, aiohttp, random
import pandas as pd

# Def des headers, bien prendre tous les headers envoyés par le browser
headers = {
    'Connection': 'keep-alive',
    'DNT': '1',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.20 Safari/537.36 Edg/97.0.1072.21',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'x-nba-stats-origin': 'stats',
    'sec-ch-ua-platform': '"Windows"',
    'Origin': 'https://www.nba.com',
    'Sec-Fetch-Site': 'same-site',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://www.nba.com/',
    'Accept-Language': 'fr,fr-FR;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'Accept-Encoding': 'gzip, deflate, br',
    'Host': 'stats.nba.com'
}

# Headers pouvant aider au rate-limit
rate_limit_bypass_headers = {
    'X-Originating-IP': '127.0.0.1',
    'X-Remote-IP': '127.0.0.1',
    'X-Remote-Addr': '127.0.0.1',
    'X-Client-IP': '127.0.0.1',
    'X-Host': '127.0.0.1',
    'X-Forwarded': '127.0.0.1',
    'X-Forwarded-By': '127.0.0.1',
    'X-Forwarded-For': '',
    'X-Forwarded-For': '127.0.0.1',
    'X-Forwarder-For': '127.0.0.1',
    'X-Forwarder-For-Original': '127.0.0.1',
    'Forwarded-For': '127.0.0.1',
    'X-Custom-IP-Authorization': '127.0.0.1',
    'X-Originating-IP': '127.0.0.1',
    'X-Remote-IP': '127.0.0.1',
    'X-Remote-Addr': '127.0.0.1',
}

magic_chars = ['%00', '%2e', '%09', '%20', '%0', '%0d', '%0a', '%0C']
platforms = ['Android', 'Windows', 'Linux', 'macOS']
user_agents_df = pd.read_csv('../data/user_agents.csv')
lang = ['fr', 'zh', 'en', 'de', 'ja', 'es', 'en-US', 'fr-CH', 'fr']
mobile = ['?0', '?1']

leaguedashplayerstats_params = (
    ('College', ''),
    ('Conference', ''),
    ('Country', ''),
    ('DateFrom', ''),
    ('DateTo', ''),
    ('Division', ''),
    ('DraftPick', ''),
    ('DraftYear', ''),
    ('GameScope', ''),
    ('GameSegment', ''),
    ('Height', ''),
    ('LastNGames', '0'),
    ('LeagueID', '00'),
    ('Location', ''),
    ('MeasureType', 'Advanced'),
    ('Month', '0'),
    ('OpponentTeamID', '0'),
    ('Outcome', ''),
    ('PORound', '0'),
    ('PaceAdjust', 'N'),
    ('PerMode', 'PerGame'),
    ('Period', '0'),
    ('PlayerExperience', ''),
    ('PlayerPosition', ''),
    ('PlusMinus', 'N'),
    ('Rank', 'N'),
    ('Season', '2021-22'),
    ('SeasonSegment', ''),
    ('SeasonType', 'Regular Season'),
    ('ShotClockRange', ''),
    ('StarterBench', ''),
    ('TeamID', '0'),
    ('TwoWay', '0'),
    ('VsConference', ''),
    ('VsDivision', ''),
    ('Weight', ''),
)

leaguedashplayerstats_params = dict(leaguedashplayerstats_params)

def rand_headers():
    rand_lang = f"{random.choice(lang)};q={random.choice(magic_chars)},{random.choice(lang)};q={random.choice(magic_chars)}"
    rand_platform = random.choice(platforms)
    rand_agent = user_agents_df.sample().values[0][0]
    rand_mobile = random.choice(mobile)
    rand_header_name, rand_header_val = random.choice(list(rate_limit_bypass_headers.items()))

    # Set Connection to close instead of keep-alive
    #HTTP keep-alive allows the same connection to be used for multiple requests, which can improve performance, especially over HTTPS, where the connection handshake can be lengthy. But the server may have a limit as to how long it's willing to keep the connection alive or how many requests it will accept on the same connection. When the limit runs out, it closes the connection.
    # rand_headers = {
    #     'Connection': 'close',
    #     'DNT': f'{random.choice([0,1])}',
    #     'sec-ch-ua-mobile': f'{rand_mobile}',
    #     'User-Agent': f'{rand_agent}',
    #     'Accept': 'application/json, text/plain, */*',
    #     'x-nba-stats-token': 'true',
    #     'x-nba-stats-origin': 'stats',
    #     'sec-ch-ua-platform': f'{rand_platform}',
    #     'Origin': 'https://www.nba.com',
    #     'Sec-Fetch-Site': 'same-site',
    #     'Sec-Fetch-Mode': 'cors',
    #     'Sec-Fetch-Dest': 'empty',
    #     'Referer': 'https://www.nba.com/',
    #     'Accept-Language': f'{rand_lang}',
    #     rand_header_name: rand_header_val,
    # }

    rand_headers = {
        'Connection': 'keep-alive',
        'DNT': f'{random.choice([0,1])}',
        'sec-ch-ua-mobile': f'{rand_mobile}',
        'User-Agent': f'{rand_agent}',
        'Accept': 'application/json, text/plain, */*',
        'x-nba-stats-token': 'true',
        'x-nba-stats-origin': 'stats',
        'sec-ch-ua-platform': f'{rand_platform}',
        'Origin': 'https://www.nba.com',
        'Sec-Fetch-Site': 'same-site',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': 'https://www.nba.com/',
        'Accept-Language': 'fr,fr-FR;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
        'Accept-Encoding': 'gzip, deflate, br',
        'Host': 'stats.nba.com'
    }

    # print(headers)
    # print("----------")
    # print(rand_headers)

    return rand_headers

def gen_seasons():
    seasons = []
    end = (2021,22)
    start = (1996,97)
    i = start[0]
    j = start[1]
    while i <= end[0]:
        if j % 100 == 0:
            j = 0
        season = f"{i}-{j}"    
        if j <= 9:
            season = f"{i}-0{j}"
        seasons.append(season)
        i+=1
        j+=1

    return seasons

seasons_dates = gen_seasons()
players_stats_by_seasons = []

In [ ]:
print('header aléatoire :')
print(rand_headers())

print('saisons : ')
print(gen_seasons())

In [ ]:
# Effectue une req

seasons_dates = gen_seasons()
players_stats_by_seasons = []

async def get(session, url, params):
    async with session.head(url, params=params) as r:
        #print(r)
        return await r.text("utf-8")

async def scrape_player_stats_by_season():
    url = 'https://stats.nba.com/stats/leaguedashplayerstats'
    params = leaguedashplayerstats_params
    async with aiohttp.ClientSession(headers=rand_headers()) as s:
        for date in seasons_dates:
            params['Season'] = date
            #print(params)
            #params.update(params)
            async with s.get(url, params=params) as r:
                print("bruh")
                r = await r.text("utf-8")
                #print(r)
                print(f"GOT {date}")
                #await asyncio.sleep(0.5)
                #Timeout of 1s : 1m18, no timeout 52s
            #r = await get(s, url, params=params)
            #print(r)
            #print(r.status)
            #text = await r.text("utf-8")
            #time.sleep(2)
            #print(r)
            #print(len(players_stats_by_seasons))
            #df = pd.DataFrame(r['resultSets'][0]['rowSet'])
            #print(df)
            #print(date)
            #print(params['Season'])
            #players_stats_by_seasons.append((date,r))
            #print(r['resultSets'][0]['rowSet'][0])
    print(players_stats_by_seasons)

            
await asyncio.gather(scrape_player_stats_by_season())

In [ ]:
# Still rate limits

connector = aiohttp.TCPConnector(limit=1, force_close=True)

async def get(session, url, headers, params):
	async with session.get(url=url, headers=headers, params=params, cookies=session.cookie_jar.filter_cookies('https://.nba.com'), proxy=proxies[0]) as response:
		cookies = session.cookie_jar.filter_cookies('https://.nba.com')
		for key, cookie in cookies.items():
			cookie = {cookie.key: cookie.value}
			print(cookie)
		return await response.text()

async def logic():
	async with aiohttp.ClientSession(connector=connector) as session:
		for param in season_params:	
			texts.append(await asyncio.gather(get(session, url, rand_headers(), param))	)
			print(f"{len(texts)}/{len(season_params)}")
			#return text

async def get_all(url):
	semaphore = asyncio.Semaphore(1)
	texts = []
	rate_limited = False
	session = aiohttp.ClientSession(connector=connector, cookie_jar=aiohttp.CookieJar())
	async with semaphore:
		while True:
			try :
				for param in season_params:	
					#if len(texts) > 0:
					#if param == season_params[len(texts)]:
						#print(f"Continuing from {len(texts)}/{len(season_params)}...")
					texts.append(await asyncio.gather(get(session, url, headers, param)))
						#texts.append(await asyncio.ensure_future(get(session, url, rand_headers(), param)))
					
					print(f"{len(texts)}/{len(season_params)}")
						
			except aiohttp.ServerDisconnectedError:
				rate_limited = True
				print("Rate-limited, sleeping for 10s...")
				await asyncio.sleep(10)

seasons_dates = gen_seasons()

# Generate parameters for each season
season_params = [(leaguedashplayerstats_params | {'Season': f"{date}"}) for date in seasons_dates]

async def main():
	#await asyncio.gather(fetch_all(urls))
	await asyncio.gather(get_all("https://stats.nba.com/stats/leaguedashplayerstats"))

await(main())